In [1]:
import pandas as pd
import tabml.datasets
from sklearn.model_selection import train_test_split

df_dict = tabml.datasets.download_movielen_1m()
users, movies, ratings = df_dict["users"], df_dict["movies"], df_dict["ratings"]

train_ratings, validation_ratings = train_test_split(
    ratings, test_size=0.1, random_state=42
)

In [2]:
users_in_validation = validation_ratings["UserID"].unique()
all_users = users["UserID"].unique()

print(f"There are {len(users_in_validation)} users in validation set.")
print(f"Total number of users: {len(all_users)}")

There are 5970 users in validation set.
Total number of users: 6040


In [3]:
movie_index_by_id = {id: i for i, id in enumerate(movies["MovieID"])}

In [4]:
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]
genre_index_by_name = {name:i for i, name in enumerate(genres)}

import numpy as np
# build binary array for movie genres
movie_features = np.zeros((len(movies), len(genres)))
for i, movie_genres in enumerate(movies["Genres"]):
    for genre in movie_genres.split("|"):        
        genre_index = genre_index_by_name[genre]
        movie_features[i, genre_index] = 1

In [5]:
from sklearn.linear_model import Ridge


def train_user_model(user_id):
    user_ratings = train_ratings[train_ratings["UserID"] == user_id]
    movie_indexes = [
        movie_index_by_id[movie_id] for movie_id in user_ratings["MovieID"]
    ]
    train_data = movie_features[movie_indexes]
    train_label = user_ratings["Rating"]
    model = Ridge(alpha=0.1)
    model.fit(train_data, train_label)
    return model


# build model for each user
user_model_dict = {}
for user_id in users["UserID"].unique():
    user_model_dict[user_id] = train_user_model(user_id)

In [6]:
def predict(user_id, movie_id):
    movie_feature = movie_features[movie_index_by_id[movie_id]].reshape((1, -1))
    pred = user_model_dict[user_id].predict(movie_feature)
    return min(max(pred, 1), 5)

In [7]:
from sklearn.metrics import mean_squared_error

def eval_rmse(ratings: pd.DataFrame) -> float:
    predictions = np.zeros(len(ratings))
    for index, row in enumerate(ratings.itertuples(index=False)):
        predictions[index] = predict(row[0], row[1])
    rmse = mean_squared_error(ratings["Rating"], predictions, squared=False)
    return rmse
    
print(f"RMSE train: {eval_rmse(train_ratings)}")
print(f"RMSE validation: {eval_rmse(validation_ratings)}")
    

RMSE train: 0.9298923151317949


RMSE validation: 1.0423634495127465


In [8]:
user_id = 160
for genre, coef in zip(genres, user_model_dict[user_id].coef_):
    print("{:15s}: {:.3f}".format(genre, coef))

Action         : -0.308
Adventure      : 0.710
Animation      : 0.198
Children's     : -0.199
Comedy         : -0.712
Crime          : 0.332
Documentary    : 0.000
Drama          : 0.225
Fantasy        : -0.396
Film-Noir      : 0.000
Horror         : 0.050
Musical        : 0.198
Mystery        : 0.000
Romance        : 0.771
Sci-Fi         : -0.373
Thriller       : -0.838
War            : -1.439
Western        : 0.000


In [9]:
user_model_dict[user_id].intercept_

3.782891147446579